In [ ]:
import torch

print("torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device available")
print("CUDA device count:", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device() if torch.cuda.is_available() else "No current CUDA device")
print("CUDA device capability:", torch.cuda.get_device_capability(0) if torch.cuda.is_available() else "No device capability available")
print("CUDA device properties:", torch.cuda.get_device_properties(0) if torch.cuda.is_available() else "No device properties available")
print("BF16 support:", torch.cuda.is_bf16_supported() if torch.cuda.is_available() else "BF16 support not available")
print("Architecture list:", torch.cuda.get_arch_list() if torch.cuda.is_available() else "No architecture list available")


torch version: 2.7.1+cu118
CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3070 Ti Laptop GPU
CUDA device count: 1
Current CUDA device: 0
CUDA device capability: (8, 6)
CUDA device properties: _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070 Ti Laptop GPU', major=8, minor=6, total_memory=8191MB, multi_processor_count=46, uuid=fefe497f-e5be-51e3-ecb6-db968d4da032, L2_cache_size=4MB)
BF16 support: True
Architecture list: ['sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_37', 'sm_90']
